In [13]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import itertools

import copy
import random, math
import os


In [14]:
import math
import numpy as np
import pexpect
import ctypes

class Connect4Solver:
    !pip install stable_baselines3
    !git clone https://github.com/TonyCongqianWang/connect4_solver_fork.git && cd connect4_solver_fork && make
    !curl -L https://github.com/PascalPons/connect4/releases/download/book/7x6.book --output 7x6.book
    solver_path='./connect4_solver_fork/c4solver_c_interface.so'
    solver_lib = ctypes.CDLL(solver_path)
            
    solver_lib.solver_init.argtypes = [ctypes.c_char_p]
    solver_lib.solver_init.restype = ctypes.POINTER(ctypes.c_void_p)
    
    solver_lib.solver_delete.argtypes = [ctypes.POINTER(ctypes.c_void_p)]
    solver_lib.solver_delete.restype = None
    
    solver_lib.solver_solve.argtypes = [ctypes.POINTER(ctypes.c_void_p), ctypes.c_char_p, ctypes.c_bool, ctypes.c_bool, ctypes.c_char_p, ctypes.c_size_t]
    solver_lib.solver_solve.restype = ctypes.c_char_p
    def __init__(self):
        """
        Initializes the Connect4Solver with the path to the solver executable.

        Args:
            solver_path (str): Path to the Connect4 solver executable.
        """
        self.MAX_SCORE = 24
        self.handle = Connect4Solver.solver_lib.solver_init(None)
        self.result_buffer = ctypes.create_string_buffer(256)

    def __del__(self):
        """
        Destructor that sends EOF to the solver process.
        """
        if hasattr(self, 'child') and self.child is not None:
            try:
                self.child.sendeof()
            except:
                pass

    def _process_output(self, prompt_str, answer_str):
        """
        Processes the output from the solver.

        Args:
            prompt_str (str): The prompt string.
            answer_str (str): The answer string.

        Returns:
            list: List of floats representing the processed output.
        """
        if answer_str.startswith(prompt_str):
            answer_str = answer_str[len(prompt_str):].strip()
            
        answer_list = [float(x) for x in answer_str.split()]
        return answer_list

    def _softmax(self, x, temperature=1.0):
        """
        Calculates a modified softmax that approaches argmax for small temperatures.

        For very small temperatures, indices with the maximum value will receive
        equal probability, and the rest will receive 0.

        Args:
            x (list): List of values.
            temperature (float): Temperature parameter for softmax.

        Returns:
            list: List of probabilities.
        """
        if temperature <= 1e-5:  # Consider a very small temperature as argmax
            max_val = max(x)
            max_indices = [i for i, val in enumerate(x) if val == max_val]
            probabilities = [0.0] * len(x)
            prob = 1.0 / len(max_indices)
            for i in max_indices:
                probabilities[i] = prob
            return probabilities
        else:
            e_x = []
            for i in x:
                # Clipping to prevent overflow for large positive values
                exponent = i / temperature
                if exponent > 100:  # Or a suitable large value
                    e_x.append(float('inf'))
                elif exponent < -100:
                    e_x.append(0.0)
                else:
                    e_x.append(math.exp(exponent))

            sum_e_x = sum(e_x)
            if sum_e_x == 0:
                return ([1.0] * len(x)) / len(x)
            return [e / sum_e_x for e in e_x]

    def _transform_and_softmax(self, data, score_offset, temperature):
        """
        Transforms and calculates the softmax of the data.

        Args:
            data (list): List of data values.
            temperature (float): Temperature parameter for softmax.

        Returns:
            list: List of softmax probabilities.
        """
        transformed_data = []
        for x in data:
            sign = 1 if x > 0 else -1 if x < 0 else 0
            if x > -1000:
                transformed_x = sign * ((abs(x) + score_offset) / self.MAX_SCORE * 5)
            else:
                transformed_x = -1000
            transformed_data.append(transformed_x)
        #print(transformed_data)
        return self._softmax(transformed_data, temperature)

    def _random_index(self, softmax_probs):
        """
        Selects a random index based on softmax probabilities.

        Args:
            softmax_probs (list): List of softmax probabilities.

        Returns:
            int: Selected index.
        """
        selected_index = np.random.choice(len(softmax_probs), p=softmax_probs)
        return selected_index

    def get_solver_move(self, move_str, temperature=1.0):
        """
        Gets a move from the solver.

        Args:
            move_str (str): Move string to send to the solver.
            temperature (float): Temperature parameter for softmax.

        Returns:
            int: Selected move index.
        """
        try:
            result = Connect4Solver.solver_lib.solver_solve(self.handle, move_str.encode("utf-8"), False, True, self.result_buffer, 256)
            answer = result.decode()
            score_offset = math.floor(len(move_str) / 2)
            probas = self._transform_and_softmax(self._process_output(move_str, answer), score_offset, temperature)
            #print(f"{answer}")
            #print(probas)
            return self._random_index(probas)
        except Exception as e:
            print(f"{e}")
            print(f"{answer}")
        return 0

Schwerwiegend: Zielpfad 'connect4_solver_fork' existiert bereits und ist kein leeres Verzeichnis.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 32.0M  100 32.0M    0     0  22.9M      0  0:00:01  0:00:01 --:--:-- 28.9M


In [15]:
class ConnectFourEnv(gym.Env):
    metadata = {"render_modes": ["human", "ansi", "rgb_array"], "render_fps": 1}
    def __init__(self, render_mode=None, board_rows=6, board_cols=7):
        super(ConnectFourEnv, self).__init__()
        self.board_rows = board_rows
        self.board_cols = board_cols
        self.action_space = spaces.Discrete(self.board_cols)  # Columns to drop a piece
        self.observation_space = spaces.Box(low=0, high=255, shape=(2, self.board_rows, self.board_cols), dtype=np.uint8)  # two binary matrices. one for each players stones
        self.render_mode = render_mode
        self.move_history = ""
        
        self.reset()

    def reset(self, seed=None, options=None):
        super().reset(seed=seed, options=options)
        self.board = np.zeros((self.board_rows, self.board_cols), dtype=np.int8)
        self.player = 1  # Player 1 starts
        self.done = False
        self.winner = None
        self.turns = 0
        self.move_history_str = ""
        info = {}
        return self._get_observation(), info

    def step(self, action):
        if self.done:
            return self._get_observation(), 0, True, False, {}

        if not self._is_valid_move(action):
            return self._get_observation(), -50, False, False, {}

        self._drop_piece(action)
        self.move_history_str += str(action + 1)
        self.turns += 1

        if self._check_win():
            self.done = True
            self.winner = self.player
            reward = 80 + 20 * (len(self.board.flatten()) - self.turns) / len(self.board.flatten())
        elif self._check_draw():
            self.done = True
            reward = 0
        else:
            reward = 0
        self.player *= -1  # Switch players
        return  self._get_observation(), reward, self.done, False, {}

    def get_valid_moves(self):
        valid_moves = []
        for col in range(self.board_cols):
            if self._is_valid_move(col):
                valid_moves.append(col)
        return valid_moves

    def _get_observation(self):
        m, n = self.board.shape
        player_perspective = self.board * self.player
        new_array = np.zeros((2, m, n), dtype=np.uint8)
        new_array[0, :, :] = 255 * (player_perspective == 1).astype(np.uint8)
        new_array[1, :, :] = 255 * (player_perspective == -1).astype(np.uint8)
        return new_array

    def _is_valid_move(self, col):
        return self.board[0, col] == 0

    def _drop_piece(self, col):
        for row in range(self.board_rows - 1, -1, -1):
            if self.board[row, col] == 0:
                self.board[row, col] = self.player
                return

    def _check_win(self):
        # Check horizontal, vertical, and diagonal wins
        for r in range(self.board_rows):
            for c in range(self.board_cols - 3):
                if (
                    self.board[r, c] == self.board[r, c + 1] == self.board[r, c + 2] == self.board[r, c + 3] != 0
                ):
                    return True

        for c in range(self.board_cols):
            for r in range(self.board_rows - 3):
                if (
                    self.board[r, c] == self.board[r + 1, c] == self.board[r + 2, c] == self.board[r + 3, c] != 0
                ):
                    return True

        for r in range(self.board_rows - 3):
            for c in range(self.board_cols - 3):
                if (
                    self.board[r, c] == self.board[r + 1, c + 1] == self.board[r + 2, c + 2] == self.board[r + 3, c + 3] != 0
                ):
                    return True

        for r in range(3, self.board_rows):
            for c in range(self.board_cols - 3):
                if (
                    self.board[r, c] == self.board[r - 1, c + 1] == self.board[r - 2, c + 2] == self.board[r - 3, c + 3] != 0
                ):
                    return True
        return False

    def _check_draw(self):
        return np.all(self.board != 0)

    def render(self):
        board_str = ""
        board_str += "-" * (self.board_cols * 2 + 3) + "\n"
        for row in self.board:
            board_str += "| "
            for cell in row:
                if cell == 1:
                    board_str += "x "
                elif cell == -1:
                    board_str += "o "
                else:
                    board_str += "  "
            board_str += "|\n"
        board_str += "-" * (self.board_cols * 2 + 3)
        print(board_str)

In [22]:
from collections import defaultdict
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

def calculate_elo(rating_a, rating_b, score_a):
    """Calculates the new Elo ratings for two players."""
    k = 8
    q = 480
    expected_a = 1 / (1 + 10**((rating_b - rating_a) / q))
    expected_b = 1 / (1 + 10**((rating_a - rating_b) / q))
    
    new_rating_a = rating_a + k * (score_a - expected_a)
    new_rating_b = rating_b + k * ((1 - score_a) - expected_b)
    return new_rating_a, new_rating_b

def evaluate_agents(agents, num_episodes=100, solver_temps=None, deterministic_agents=False):
    """Evaluates multiple trained PPO agents and calculates Elo scores."""
    env = ConnectFourEnv()
    agent_paths = agents.keys()
    print(f"{agent_paths=}")
    agents = [model for _, model in agents.items()]
    elo_ratings = [1500] * len(agents)
    solver = Connect4Solver()
    if solver_temps is None:
        solver_temps = [0.0, 0.1, 0.15, 0.2, 0.3]
        # Empirically determined ratings
        elo_ratings += [1970, 1910, 1840, 1820, 1710]
        fixed_solver_ratings = True
    else:
        elo_ratings += [1500] * len(solver_temps)
        fixed_solver_ratings = False
    agents += solver_temps
    agent_names = [path.split("/")[-1] for path in agent_paths] + [f"Solver_{t}" for t in solver_temps]
    num_agents = len(agents)
    
    results = defaultdict(lambda: defaultdict(lambda: {"wins_first": 0, "draws_first": 0, "loses_first": 0}))
    # Generate all possible matchups
    matchups = [(i, j) for i in range(num_agents) for j in range(num_agents)]  # Add self-play

    for episode in range(num_episodes):
        random.shuffle(matchups) #shuffle matchups for each episode.
        for agent1_index, agent2_index in matchups:
            agent1 = agents[agent1_index]
            agent2 = agents[agent2_index]

            agent1_wins_first = 0
            agent1_draws_first = 0
            agent1_loses_first = 0

            obs, _ = env.reset()
            done = False

            while not done:
                current_player = env.player
                if current_player == 1:
                    if type(agent1) == float:
                        action = solver.get_solver_move(env.move_history_str, agent1)
                    else:
                        action = agent1.predict(obs, deterministic=deterministic_agents)
                else:
                    if type(agent2) == float:
                        action = solver.get_solver_move(env.move_history_str, agent2)
                    else:
                        action = agent2.predict(obs, deterministic=deterministic_agents)

                valid_moves = env.get_valid_moves()
                if action not in valid_moves:
                    action = random.choice(valid_moves)
                obs, reward, done, truncated, _ = env.step(action)

                if done or truncated:
                    if reward > 0:
                        if current_player == 1:
                            agent1_wins_first = 1
                        else:
                            agent1_loses_first = 1
                    else:
                        agent1_draws_first = 1
                    break

            # Calculate Elo. Don't change Elo for self-play or for solvers
            score_agent1 = (agent1_wins_first + 0.5 * agent1_draws_first)
            if agent1_index != agent2_index:
                new_rating_1, new_rating_2 = calculate_elo(
                    elo_ratings[agent1_index], elo_ratings[agent2_index], score_agent1
                )
                if not (type(agents[agent1_index]) == float and fixed_solver_ratings):
                    elo_ratings[agent1_index] = new_rating_1
                if not (type(agents[agent2_index]) == float and fixed_solver_ratings):
                    elo_ratings[agent2_index] = new_rating_2

            # Update results
            results[agent1_index][agent2_index]["wins_first"] += agent1_wins_first
            results[agent1_index][agent2_index]["draws_first"] += agent1_draws_first
            results[agent1_index][agent2_index]["loses_first"] += agent1_loses_first
        print(f"Episode {episode + 1} / 100 done.")

    env.close()

    sorted_agents = sorted(range(num_agents), key=lambda i: elo_ratings[i], reverse=True)

    columns = ["Agent", "Elo"] + [f"Agent {i}" for i in range(num_agents)]
    data = []

    for agent1_index in sorted_agents:
        agent_name = agent_names[agent1_index]
        row = [agent_name, f"{elo_ratings[agent1_index]:.2f}"]
        for agent2_index in sorted_agents:
            data_dict = results[agent1_index][agent2_index]
            row.append(f"{data_dict['wins_first']} / {data_dict['draws_first']} / {data_dict['loses_first']}")
        data.append(row)

    df = pd.DataFrame(data, columns=columns)
    return results, df

In [28]:
import onnxruntime as ort
import numpy as np

class ONNXModel:
    def __init__(self, model_path: str, providers: list = None):
        """
        Initialize the ONNX model.
        :param model_path: Path to the ONNX model file.
        :param providers: Optional list of execution providers (e.g., ['CPUExecutionProvider']).
        """
        self.session = ort.InferenceSession(model_path, providers=providers or ['CPUExecutionProvider'])
        self.input_name = self.session.get_inputs()[0].name
        self.input_shape = self.session.get_inputs()[0].shape
        self.input_type = self.session.get_inputs()[0].type
        self.output_names = [output.name for output in self.session.get_outputs()]
        
    def predict(self, input_array: np.ndarray, deterministic: bool=True, temperature=0.25) -> int:
        out = self.run(input_array)
        policy, *_ = out
        if deterministic:
            return np.argmax(policy)
        else:
            if not (policy >= 0).all() or not np.isclose(np.sum(policy), 1.0):
                policy = policy / temperature
                e_logits = np.exp(policy - np.max(policy))
                policy = e_logits / np.sum(e_logits)
            return np.random.choice(len(policy), p=policy)

    def run(self, input_array: np.ndarray) -> np.ndarray:
        input_array = input_array.astype(np.float32)
        if len(input_array.shape) < 4:
            input_array = np.expand_dims(input_array, axis=0)
        if not isinstance(input_array, np.ndarray):
            raise TypeError("Input must be a NumPy array")

        input_data = {self.input_name: input_array}
        outputs = self.session.run(self.output_names, input_data)
        policy, value, *extras = outputs
        policy = policy[0]
        return policy, value, *extras

In [29]:
solver_temps = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
onnx_paths = ["best_models/sb_best.onnx", "best_models/vit_medium_r0.onnx", "best_models/cvn_tiny_r0.onnx", "best_models/vit_medium_r1.onnx", "best_models/cvn_tiny_r1.onnx"]
agents = {path : ONNXModel(path) for path in onnx_paths}
table, df = evaluate_agents(agents, num_episodes=100, solver_temps=solver_temps)

agent_paths=dict_keys(['best_models/sb_best.onnx', 'best_models/vit_medium_r0.onnx', 'best_models/cvn_tiny_r0.onnx', 'best_models/vit_medium_r1.onnx', 'best_models/cvn_tiny_r1.onnx'])


Loading opening book from file: 7x6.book. done


Episode 1 / 100 done.
Episode 2 / 100 done.
Episode 3 / 100 done.
Episode 4 / 100 done.
Episode 5 / 100 done.
Episode 6 / 100 done.
Episode 7 / 100 done.
Episode 8 / 100 done.
Episode 9 / 100 done.
Episode 10 / 100 done.
Episode 11 / 100 done.
Episode 12 / 100 done.
Episode 13 / 100 done.
Episode 14 / 100 done.
Episode 15 / 100 done.
Episode 16 / 100 done.
Episode 17 / 100 done.
Episode 18 / 100 done.
Episode 19 / 100 done.
Episode 20 / 100 done.
Episode 21 / 100 done.
Episode 22 / 100 done.
Episode 23 / 100 done.
Episode 24 / 100 done.
Episode 25 / 100 done.
Episode 26 / 100 done.
Episode 27 / 100 done.
Episode 28 / 100 done.
Episode 29 / 100 done.
Episode 30 / 100 done.
Episode 31 / 100 done.
Episode 32 / 100 done.
Episode 33 / 100 done.
Episode 34 / 100 done.
Episode 35 / 100 done.
Episode 36 / 100 done.
Episode 37 / 100 done.
Episode 38 / 100 done.
Episode 39 / 100 done.
Episode 40 / 100 done.
Episode 41 / 100 done.
Episode 42 / 100 done.
Episode 43 / 100 done.
Episode 44 / 100 don

In [30]:
df

,Agent,Elo,Agent 0,Agent 1,Agent 2,Agent 3,Agent 4,Agent 5,Agent 6,Agent 7,Agent 8,Agent 9,Agent 10,Agent 11
0,Solver_0.05,1621.03,93 / 7 / 0,97 / 3 / 0,98 / 2 / 0,99 / 1 / 0,96 / 4 / 0,98 / 2 / 0,96 / 4 / 0,98 / 2 / 0,98 / 2 / 0,98 / 2 / 0,99 / 1 / 0,97 / 3 / 0
1,Solver_0.0,1617.18,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0
2,vit_medium_r1.onnx,1598.04,85 / 14 / 1,91 / 5 / 4,78 / 22 / 0,87 / 8 / 5,91 / 1 / 8,84 / 9 / 7,85 / 13 / 2,93 / 3 / 4,92 / 3 / 5,94 / 1 / 5,95 / 1 / 4,100 / 0 / 0
3,cvn_tiny_r0.onnx,1592.89,82 / 10 / 8,78 / 16 / 6,71 / 20 / 9,80 / 10 / 10,78 / 11 / 11,82 / 8 / 10,71 / 23 / 6,81 / 10 / 9,92 / 3 / 5,95 / 1 / 4,92 / 3 / 5,98 / 0 / 2
4,Solver_0.1,1591.24,85 / 14 / 1,82 / 16 / 2,80 / 19 / 1,83 / 14 / 3,81 / 15 / 4,88 / 10 / 2,86 / 13 / 1,79 / 18 / 3,82 / 17 / 1,82 / 14 / 4,87 / 12 / 1,91 / 9 / 0
5,cvn_tiny_r1.onnx,1569.49,83 / 10 / 7,79 / 13 / 8,66 / 24 / 10,89 / 6 / 5,92 / 3 / 5,81 / 10 / 9,73 / 19 / 8,94 / 4 / 2,91 / 4 / 5,92 / 3 / 5,91 / 3 / 6,98 / 0 / 2
6,vit_medium_r0.onnx,1568.67,81 / 14 / 5,75 / 13 / 12,75 / 19 / 6,90 / 2 / 8,89 / 6 / 5,80 / 6 / 14,84 / 7 / 9,94 / 4 / 2,93 / 4 / 3,93 / 3 / 4,95 / 1 / 4,100 / 0 / 0
7,Solver_0.15,1513.49,66 / 20 / 14,55 / 31 / 14,55 / 34 / 11,64 / 23 / 13,66 / 29 / 5,56 / 34 / 10,68 / 27 / 5,62 / 32 / 6,64 / 30 / 6,73 / 22 / 5,75 / 18 / 7,85 / 15 / 0
8,Solver_0.2,1451.91,49 / 31 / 20,56 / 22 / 22,43 / 31 / 26,47 / 32 / 21,37 / 39 / 24,46 / 28 / 26,50 / 30 / 20,46 / 39 / 15,51 / 34 / 15,54 / 33 / 13,70 / 24 / 6,77 / 20 / 3
9,Solver_0.25,1388.85,33 / 30 / 37,26 / 34 / 40,44 / 28 / 28,47 / 28 / 25,36 / 29 / 35,42 / 25 / 33,44 / 30 / 26,33 / 34 / 33,41 / 38 / 21,54 / 25 / 21,62 / 25 / 13,82 / 13 / 5


In [26]:
solver_temps = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
onnx_paths = ["best_models/sb_best.onnx", "best_models/vit_medium_r0.onnx", "best_models/cvn_tiny_r0.onnx", "best_models/vit_medium_r1.onnx", "best_models/cvn_tiny_r1.onnx"]
agents = {path : ONNXModel(path) for path in onnx_paths}
table, df2 = evaluate_agents(agents, num_episodes=100, solver_temps=solver_temps, deterministic_agents=True)

agent_paths=dict_keys(['best_models/sb_best.onnx', 'best_models/vit_medium_r0.onnx', 'best_models/cvn_tiny_r0.onnx', 'best_models/vit_medium_r1.onnx', 'best_models/cvn_tiny_r1.onnx'])


Loading opening book from file: 7x6.book. done


Episode 1 / 100 done.
Episode 2 / 100 done.
Episode 3 / 100 done.
Episode 4 / 100 done.
Episode 5 / 100 done.
Episode 6 / 100 done.
Episode 7 / 100 done.
Episode 8 / 100 done.
Episode 9 / 100 done.
Episode 10 / 100 done.
Episode 11 / 100 done.
Episode 12 / 100 done.
Episode 13 / 100 done.
Episode 14 / 100 done.
Episode 15 / 100 done.
Episode 16 / 100 done.
Episode 17 / 100 done.
Episode 18 / 100 done.
Episode 19 / 100 done.
Episode 20 / 100 done.
Episode 21 / 100 done.
Episode 22 / 100 done.
Episode 23 / 100 done.
Episode 24 / 100 done.
Episode 25 / 100 done.
Episode 26 / 100 done.
Episode 27 / 100 done.
Episode 28 / 100 done.
Episode 29 / 100 done.
Episode 30 / 100 done.
Episode 31 / 100 done.
Episode 32 / 100 done.
Episode 33 / 100 done.
Episode 34 / 100 done.
Episode 35 / 100 done.
Episode 36 / 100 done.
Episode 37 / 100 done.
Episode 38 / 100 done.
Episode 39 / 100 done.
Episode 40 / 100 done.
Episode 41 / 100 done.
Episode 42 / 100 done.
Episode 43 / 100 done.
Episode 44 / 100 don

In [27]:
df2

,Agent,Elo,Agent 0,Agent 1,Agent 2,Agent 3,Agent 4,Agent 5,Agent 6,Agent 7,Agent 8,Agent 9,Agent 10,Agent 11
0,vit_medium_r1.onnx,1666.04,0 / 100 / 0,0 / 100 / 0,92 / 8 / 0,92 / 8 / 0,95 / 3 / 2,100 / 0 / 0,93 / 4 / 3,100 / 0 / 0,95 / 3 / 2,100 / 0 / 0,98 / 1 / 1,100 / 0 / 0
1,vit_medium_r0.onnx,1637.05,0 / 100 / 0,0 / 100 / 0,80 / 12 / 8,82 / 15 / 3,89 / 8 / 3,100 / 0 / 0,85 / 8 / 7,100 / 0 / 0,94 / 2 / 4,93 / 4 / 3,98 / 0 / 2,100 / 0 / 0
2,Solver_0.0,1630.16,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0,100 / 0 / 0
3,Solver_0.05,1619.92,99 / 1 / 0,99 / 1 / 0,95 / 5 / 0,97 / 3 / 0,98 / 2 / 0,98 / 2 / 0,95 / 5 / 0,98 / 2 / 0,95 / 5 / 0,99 / 1 / 0,98 / 2 / 0,99 / 1 / 0
4,Solver_0.1,1589.96,82 / 15 / 3,82 / 14 / 4,82 / 17 / 1,82 / 14 / 4,80 / 17 / 3,82 / 15 / 3,69 / 28 / 3,84 / 16 / 0,79 / 19 / 2,88 / 8 / 4,84 / 16 / 0,91 / 9 / 0
5,cvn_tiny_r0.onnx,1526.84,0 / 100 / 0,0 / 100 / 0,73 / 17 / 10,74 / 18 / 8,81 / 10 / 9,100 / 0 / 0,88 / 5 / 7,100 / 0 / 0,91 / 6 / 3,94 / 4 / 2,95 / 2 / 3,100 / 0 / 0
6,Solver_0.15,1513.24,66 / 24 / 10,70 / 18 / 12,63 / 28 / 9,64 / 24 / 12,59 / 28 / 13,74 / 17 / 9,63 / 27 / 10,67 / 19 / 14,65 / 27 / 8,66 / 26 / 8,74 / 23 / 3,85 / 15 / 0
7,cvn_tiny_r1.onnx,1499.81,0 / 0 / 100,0 / 0 / 100,74 / 12 / 14,77 / 13 / 10,84 / 7 / 9,100 / 0 / 0,95 / 2 / 3,100 / 0 / 0,93 / 1 / 6,97 / 2 / 1,99 / 0 / 1,100 / 0 / 0
8,Solver_0.2,1457.30,50 / 32 / 18,59 / 23 / 18,51 / 28 / 21,43 / 39 / 18,49 / 32 / 19,59 / 23 / 18,39 / 41 / 20,72 / 13 / 15,52 / 29 / 19,60 / 23 / 17,64 / 22 / 14,77 / 22 / 1
9,Solver_0.25,1424.12,39 / 29 / 32,45 / 25 / 30,41 / 27 / 32,39 / 30 / 31,32 / 38 / 30,45 / 14 / 41,41 / 27 / 32,54 / 18 / 28,40 / 32 / 28,50 / 33 / 17,50 / 30 / 20,76 / 22 / 2
